In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
from torchvision import models
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
import old_utils_averaged_filters as old_ut_avg
import old_utils_multi_image_old_snr as old_ut_multi
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



In [2]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')


train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 30000, 3750, 3750)






train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



In [3]:
#check distribution of classes 

print("Check overlap between datasets:\n")
print("\nTrain Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("\nTrain Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("\nValidation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print()
print("\nTrain Subset Distribution:\n")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:


Train Subset and Validation Subset:
No actual data overlap detected.

Train Subset and Test Subset:
No actual data overlap detected.

Validation Subset and Test Subset:
No actual data overlap detected.


Train Subset Distribution:

Total samples in subset: 30000
Model ADM, Class nature: 1815 (6.05%)
Model ADM, Class ai: 1935 (6.45%)
Model BigGAN, Class ai: 1867 (6.22%)
Model BigGAN, Class nature: 1883 (6.28%)
Model Midjourney, Class ai: 1884 (6.28%)
Model Midjourney, Class nature: 1866 (6.22%)
Model VQDM, Class ai: 1810 (6.03%)
Model VQDM, Class nature: 1940 (6.47%)
Model glide, Class ai: 1891 (6.30%)
Model glide, Class nature: 1859 (6.20%)
Model stable_diffusion_v_1_4, Class ai: 1892 (6.31%)
Model stable_diffusion_v_1_4, Class nature: 1858 (6.19%)
Model stable_diffusion_v_1_5, Class nature: 1828 (6.09%)
Model stable_diffusion_v_1_5, Class ai: 1922 (6.41%)
Model wukong, Class nature: 1852 (6.17%)
Model wukong, Class ai: 1898 (6.33%)

Validation Subset 

In [4]:
class BaseClassifier(nn.Module):
    def __init__(self,kernels):
        super(BaseClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        self.classifier = ut.DeepClassifier() 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.classifier(feature_difference)

        return outputs

In [5]:
class ResNetClassifier(nn.Module):
    def __init__(self,kernels):
        super(ResNetClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        resnet_weights = models.ResNet50_Weights.DEFAULT
        self.resnet = models.resnet50(weights=resnet_weights)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a new classifier layer
        self.classifier = nn.Linear(self.resnet.fc.in_features, 2)
        # Adaptive pool to make sure output from feature maps is of fixed size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        
        # Process feature difference through the ResNet features
        features = self.features(feature_difference)
        pooled_features = self.adaptive_pool(features)
        flat_features = torch.flatten(pooled_features, 1)
        outputs = self.classifier(flat_features)

        return outputs

In [6]:
class SwinClassification(nn.Module):
    def __init__(self,kernels):
        super(SwinClassification, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits


## Train with freezing the pretrained models and only train the last layer with new preprocessing

In [7]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5 ]
#print model and freeze and pre
print("Training SWIN Transformer Frozen w/New Pre\n")

for learning_rate in learning_rates: 
    print(f"Training with Learning Rate for SWIN: {learning_rate} \n\n")   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_frozen.pth'

    #freeze the transformer layers
    for param in model.transformer.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.transformer.classifier.parameters():
        param.requires_grad = True

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training SWIN Transformer Frozen w/New Pre

Training with Learning Rate for SWIN: 0.001 




You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.3895, Train Acc: 0.8254, Val Loss: 0.2093, Val Acc: 0.9279

Saved new best model with accuracy: 0.9279


Epoch 2/10
,Train Loss: 0.2740, Train Acc: 0.8868, Val Loss: 0.1849, Val Acc: 0.9290

Saved new best model with accuracy: 0.9290


...

Epoch 6/10
,Train Loss: 0.5664, Train Acc: 0.7170, Val Loss: 0.5127, Val Acc: 0.7660



Epoch 7/10
,Train Loss: 0.5549, Train Acc: 0.7291, Val Loss: 0.4996, Val Acc: 0.7762



Epoch 8/10
,Train Loss: 0.5444, Train Acc: 0.7331, Val Loss: 0.4886, Val Acc: 0.7861



Epoch 9/10
,Train Loss: 0.5324, Train Acc: 0.7440, Val Loss: 0.4753, Val Acc: 0.7898



Epoch 10/10
,Train Loss: 0.5260, Train Acc: 0.7481, Val Loss: 0.4633, Val Acc: 0.7959



In [8]:
#ResNET
print("Training RES Frozen w/New Pre\n")
learning_rates = [1e-3,1e-4, 1e-5 ]
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_RESNET_frozen.pth'

    #freeze resnet layers
    for param in model.resnet.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training RES Frozen w/New Pre

Training with Learning Rate: 0.001 




No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.4085, Train Acc: 0.8196, Val Loss: 0.2993, Val Acc: 0.8806

Saved new best model with accuracy: 0.8806


Epoch 2/10
,Train Loss: 0.3082, Train Acc: 0.8740, Val Loss: 0.2711, Val Acc: 0.8905

Saved new best model with accuracy: 0.8905


Epoch 3/10
,Train Loss: 0.2818, Train Acc: 0.8846, Val Loss: 0.2419, Val Acc: 0.9095

Saved new best model with accuracy: 0.9095


...

Epoch 6/10
,Train Loss: 0.4905, Train Acc: 0.7862, Val Loss: 0.4731, Val Acc: 0.7970



Epoch 7/10
,Train Loss: 0.4766, Train Acc: 0.7923, Val Loss: 0.4604, Val Acc: 0.8015



Epoch 8/10
,Train Loss: 0.4682, Train Acc: 0.7959, Val Loss: 0.4463, Val Acc: 0.8120



Epoch 9/10
,Train Loss: 0.4573, Train Acc: 0.8017, Val Loss: 0.4380, Val Acc: 0.8149



Epoch 10/10
,Train Loss: 0.4494, Train Acc: 0.8040, Val Loss: 0.4301, Val Acc: 0.8216



## Unfreeze the pretrained model and Original Model with new preprocessing

In [9]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training SWIN Transformer UNFrozen w/New Pre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_unfrozen.pth'



    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training SWIN Transformer UNFrozen w/New Pre

Training with Learning Rate: 0.001 




You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.3782, Train Acc: 0.8329, Val Loss: 0.6932, Val Acc: 0.5061

Saved new best model with accuracy: 0.5061


Epoch 2/10
,Train Loss: 0.6941, Train Acc: 0.5008, Val Loss: 0.6941, Val Acc: 0.5061



...

Epoch 6/10
,Train Loss: 0.1519, Train Acc: 0.9410, Val Loss: 0.0843, Val Acc: 0.9720



Epoch 7/10
,Train Loss: 0.1423, Train Acc: 0.9449, Val Loss: 0.0738, Val Acc: 0.9730



Epoch 8/10
,Train Loss: 0.1297, Train Acc: 0.9498, Val Loss: 0.0764, Val Acc: 0.9736



Epoch 9/10
,Train Loss: 0.1203, Train Acc: 0.9542, Val Loss: 0.0854, Val Acc: 0.9688



Epoch 10/10
,Train Loss: 0.1104, Train Acc: 0.9586, Val Loss: 0.0583, Val Acc: 0.9781



In [10]:
#ResNET
print("Training RES UNFrozen w/New Pre\n")

for learning_rate in learning_rates:
    
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_RESNET_unfrozen.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training RES UNFrozen w/New Pre

Training with Learning Rate: 0.001 




No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.1411, Train Acc: 0.9477, Val Loss: 0.1070, Val Acc: 0.9663

Saved new best model with accuracy: 0.9663


Epoch 2/10
,Train Loss: 0.0623, Train Acc: 0.9787, Val Loss: 0.0447, Val Acc: 0.9858

Saved new best model with accuracy: 0.9858


Epoch 3/10
,Train Loss: 0.0520, Train Acc: 0.9824, Val Loss: 0.0261, Val Acc: 0.9920

Saved new best model with accuracy: 0.9920


...

Epoch 6/10
,Train Loss: 0.3192, Train Acc: 0.8746, Val Loss: 0.2727, Val Acc: 0.9068



Epoch 7/10
,Train Loss: 0.2865, Train Acc: 0.8873, Val Loss: 0.2390, Val Acc: 0.9137



Epoch 8/10
,Train Loss: 0.2623, Train Acc: 0.8969, Val Loss: 0.2233, Val Acc: 0.9153



Epoch 9/10
,Train Loss: 0.2392, Train Acc: 0.9065, Val Loss: 0.2049, Val Acc: 0.9252



Epoch 10/10
,Train Loss: 0.2228, Train Acc: 0.9128, Val Loss: 0.1898, Val Acc: 0.9314



In [11]:
#original model
print("Training BASE w/New Pre\n")

for learning_rate in learning_rates:
    
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = BaseClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPre_Base.pth'
    
    
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training BASE w/New Pre

Training with Learning Rate: 0.001 


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.4081, Train Acc: 0.8161, Val Loss: 0.1967, Val Acc: 0.9220

Saved new best model with accuracy: 0.9220


Epoch 2/10
,Train Loss: 0.2205, Train Acc: 0.9177, Val Loss: 0.3419, Val Acc: 0.8365



Epoch 3/10
,Train Loss: 0.1757, Train Acc: 0.9380, Val Loss: 0.1314, Val Acc: 0.9605

Saved new best model with accuracy: 0.9605


Epoch 4/10
,Train Loss: 0.1465, Train Acc: 0.9471, Val Loss: 0.1205, Val Acc: 0.9623

Saved new best model with accuracy: 0.9623


...

Epoch 6/10
,Train Loss: 0.5957, Train Acc: 0.6866, Val Loss: 0.5650, Val Acc: 0.7230



Epoch 7/10
,Train Loss: 0.5839, Train Acc: 0.6977, Val Loss: 0.5637, Val Acc: 0.7305



Epoch 8/10
,Train Loss: 0.5702, Train Acc: 0.7109, Val Loss: 0.5378, Val Acc: 0.7479



Epoch 9/10
,Train Loss: 0.5567, Train Acc: 0.7265, Val Loss: 0.5253, Val Acc: 0.7604



Epoch 10/10
,Train Loss: 0.5425, Train Acc: 0.7373, Val Loss: 0.5158, Val Acc: 0.7687

